loading libraries

In [57]:
import numpy as np
import json as js
import tensorflow as tf
from keras import models
from keras.utils import register_keras_serializable #type: ignore
import cv2
from os import listdir as ld
from os import path
from tqdm import tqdm

Loading paths

In [58]:
with open("config_CNN.json",'r') as file:
    paths = js.load(file)

loading model

In [59]:
def compute_iou_tf(box1, box2):
    box1_x1 = box1[:, 0]
    box1_y1 = box1[:, 1]
    box1_x2 = box1[:, 0] + box1[:, 2]
    box1_y2 = box1[:, 1] + box1[:, 3]
    box2_x1 = box2[:, 0]
    box2_y1 = box2[:, 1]
    box2_x2 = box2[:, 0] + box2[:, 2]
    box2_y2 = box2[:, 1] + box2[:, 3]
    x1 = tf.maximum(box1_x1, box2_x1)
    y1 = tf.maximum(box1_y1, box2_y1)
    x2 = tf.minimum(box1_x2, box2_x2)
    y2 = tf.minimum(box1_y2, box2_y2)
    intersection = tf.maximum(0.0, x2 - x1) * tf.maximum(0.0, y2 - y1)
    area1 = box1[:, 2] * box1[:, 3]
    area2 = box2[:, 2] * box2[:, 3]
    union = area1 + area2 - intersection
    return tf.math.divide_no_nan(intersection, union)

In [60]:
@register_keras_serializable()
class IoUMetric(tf.keras.metrics.Metric):
    def __init__(self, name='iou_metric', **kwargs):
        super(IoUMetric, self).__init__(name=name, **kwargs)
        self.total_iou = self.add_weight(name='total_iou', initializer='zeros')
        self.count = self.add_weight(name='count', initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.reshape(y_true, (-1, 4))
        y_pred = tf.reshape(y_pred, (-1, 4))

        ious = compute_iou_tf(y_true, y_pred)
        self.total_iou.assign_add(tf.reduce_sum(ious))
        self.count.assign_add(tf.cast(tf.size(ious), tf.float32))

    def result(self):
        return tf.math.divide_no_nan(self.total_iou, self.count)

    def reset_states(self):
        self.total_iou.assign(0.0)
        self.count.assign(0.0)


In [61]:
model = models.load_model(paths["Trained_model"], custom_objects={"IoUMetric": IoUMetric})

Evaluation on validation data

In [62]:
def compute_iou(box1, box2):
    x1_min = box1[0]
    y1_min = box1[1]
    x1_max = box1[0] + box1[2]
    y1_max = box1[1] + box1[3]
    x2_min = box2[0]
    y2_min = box2[1]
    x2_max = box2[0] + box2[2]
    y2_max = box2[1] + box2[3]
    inter_xmin = max(x1_min, x2_min)
    inter_ymin = max(y1_min, y2_min)
    inter_xmax = min(x1_max, x2_max)
    inter_ymax = min(y1_max, y2_max)
    inter_area = max(0, inter_xmax - inter_xmin) * max(0, inter_ymax - inter_ymin)
    box1_area = (x1_max - x1_min) * (y1_max - y1_min)
    box2_area = (x2_max - x2_min) * (y2_max - y2_min)
    union_area = box1_area + box2_area - inter_area
    return inter_area / union_area if union_area > 0 else 0

TP, FP, FN = 0, 0, 0
iou_threshold = 0.000001

In [63]:
val_data_path = paths["Validation_resized"]

with open(paths["Preprocessed_Validation_json"],'r') as file:
    val_json = js.load(file)

gt_boxes_dict = {item["img_id"]: item["bbox"] for item in val_json}

for fname in tqdm(ld(val_data_path)):
    if fname.endswith((".jpg", ".png", ".jpeg")):
        if fname not in gt_boxes_dict:
            continue

        img_path = path.join(val_data_path, fname)
        img = cv2.imread(img_path)
        img_input = np.expand_dims(img, axis=0)  
        pred = model.predict(img_input, verbose=0)  
        gt_box = gt_boxes_dict[fname] 
        iou = compute_iou(pred['bbox_output'][0], gt_box)

        if iou >= iou_threshold:
            TP += 1
            
        else:
            FP += 1
            FN += 1

100%|██████████| 5000/5000 [05:46<00:00, 14.43it/s]


In [64]:
precision = TP / (TP + FP + 1e-6)
recall = TP / (TP + FN + 1e-6)
f1_score = 2 * precision * recall / (precision + recall + 1e-6)
print(f"\n[Evaluation Results]")
print(f"True Positives : {TP}")
print(f"False Positives: {FP}")
print(f"False Negatives: {FN}")
print(f"Precision      : {precision:.4f}")
print(f"Recall         : {recall:.4f}")
print(f"F1 Score       : {f1_score:.4f}")


[Evaluation Results]
True Positives : 290
False Positives: 4662
False Negatives: 4662
Precision      : 0.0586
Recall         : 0.0586
F1 Score       : 0.0586
